In [1]:
import pandas as pd
import base64
from openai import OpenAI
import json
from pydantic import BaseModel
import cv2
import numpy as np
import fitz
from PIL import Image
from io import BytesIO
pd.set_option('display.max_columns', None)
from typing import Optional
import matplotlib.pyplot as plt
import json


import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Alignment, Border, Side, Font

In [2]:
def get_contours(image_path):
    # Загрузка изображения
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Инвертируем изображение (пунктирные линии должны быть белыми)
    _, binary = cv2.threshold(image, 150, 255, cv2.THRESH_BINARY_INV)

    # Применяем размытие, чтобы убрать шум
    blurred = cv2.GaussianBlur(binary, (5, 5), 0)

    # Используем морфологию для выделения прямоугольных структур
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    processed = cv2.morphologyEx(blurred, cv2.MORPH_CLOSE, kernel, iterations=1)

    # Поиск контуров
    contours, _ = cv2.findContours(processed, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=lambda x: x[0][0][1])
    contours = sorted(contours, key=lambda x: x[0][0][0])
    contours = sorted(contours, key=lambda c: (cv2.boundingRect(c)[1], cv2.boundingRect(c)[0]))

    # Копия изображения для отображения результатов
    output = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

    # Фильтруем и выделяем только прямоугольные блоки
    contours_valid = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        # Условие для фильтрации блоков по размеру
        if 100 < w < 150 and 100 < h < 200:  # Подстраивайте размеры под бланк
            contours_valid.append(contour)
            cv2.rectangle(output, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    return contours_valid, output

def visualize_row(contours, image_path):
    # Копия изображения для отображения результатов
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    output = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

    # Фильтруем и выделяем только прямоугольные блоки
    contours_valid = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        # Условие для фильтрации блоков по размеру
        if 100 < w < 150 and 100 < h < 200:  # Подстраивайте размеры под бланк
            contours_valid.append(contour)
            cv2.rectangle(output, (x, y), (x + w, y + h), (0, 255, 0), 2)

    return output
    

In [3]:
numb_cells_dict = {'subject':10,
                   'user_id':8,
                   'version':4,
                   'answer1':9,
                   'correction1':9,
                   'answer2':9,
                   'correction2':9,
                   'answer3':9,
                   'correction3':9,
                   'answer4':9,
                   'correction4':9,
                   'answer5':9,
                   'correction5':9,
                   'answer6':9,
                   'correction6':9,
                   'answer7':9,
                   'correction7':9,
                   'answer8':9,
                   'correction8':9,
                   'answer9':9,
                   'correction9':9,
                   'answer10':9,
                   'correction10':9}

contours, output = get_contours("test_pics/empty_example_cells.jpg")


rows_dict = {}
j= 0 
for key, value in numb_cells_dict.items():
    rows_dict[key] = dict() 
    for i in range(value):
        rows_dict[key][i] = contours[j]
        j+=1

# img_to_align = 'test_pics/empty_example.jpg'
# for key in rows_dict.keys():
#     output1 = visualize_row(rows_dict[key].values(), img_to_align)
#     plt.figure(figsize=(10, 10))  # Make the plot bigger
#     plt.imshow(output1, cmap='gray')
#     plt.show()


In [4]:
import json

# Convert rows_dict to JSON format
json_rows_dict = json.dumps(rows_dict, default=lambda x: x.tolist())

# Save the JSON to a file
with open('rows_data.json', 'w') as json_file:
    json_file.write(json_rows_dict)

In [5]:
rows_json = json.loads(json_rows_dict)

In [6]:
with open('rows_data.json', 'r') as json_file:
    rows_json = json.load(json_file)



In [7]:
rows_json

{'subject': {'0': [[[1008, 366]],
   [[1009, 365]],
   [[1141, 365]],
   [[1142, 366]],
   [[1142, 510]],
   [[1141, 511]],
   [[1009, 511]],
   [[1008, 510]]],
  '1': [[[1148, 366]],
   [[1149, 365]],
   [[1281, 365]],
   [[1282, 366]],
   [[1282, 510]],
   [[1281, 511]],
   [[1265, 511]],
   [[1264, 510]],
   [[1260, 510]],
   [[1259, 511]],
   [[1236, 511]],
   [[1235, 510]],
   [[1231, 510]],
   [[1230, 511]],
   [[1222, 511]],
   [[1221, 510]],
   [[1217, 510]],
   [[1216, 511]],
   [[1193, 511]],
   [[1192, 510]],
   [[1188, 510]],
   [[1187, 511]],
   [[1164, 511]],
   [[1163, 510]],
   [[1159, 510]],
   [[1158, 511]],
   [[1150, 511]],
   [[1149, 510]],
   [[1149, 505]],
   [[1148, 504]],
   [[1148, 496]],
   [[1149, 495]],
   [[1149, 491]],
   [[1148, 490]],
   [[1148, 467]],
   [[1149, 466]],
   [[1149, 462]],
   [[1148, 461]],
   [[1148, 438]],
   [[1149, 437]],
   [[1149, 433]],
   [[1148, 432]],
   [[1148, 424]],
   [[1149, 423]],
   [[1149, 419]],
   [[1148, 418]],
   [[1